**Introduction**

Basic AI Writing Assistant Agent

*   Designing and implementing an AI-powered agent that assists users with writing tasks.
*   The agent helps to improve the quality of text by performing tasks such as grammar correction, vocabulary enhancement, and sentence rewriting.







Installing necessary packages

In [ ]:
!pip install langchain langgraph langsmith langchain-groq langchain_community

Importing necessary libraries

In [ ]:
import re
from getpass import getpass
from typing import List, Optional, TypedDict
from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq

Setting up LLM with api

In [ ]:
from google.colab import userdata
groq_api_key= userdata.get('GROQ_API_KEY')

In [ ]:
MODEL_NAME = "deepseek-r1-distill-llama-70b"

llm = ChatGroq(
    model=MODEL_NAME,
    temperature=0.0,
    max_tokens=None,
    groq_api_key=groq_api_key
)
print("ChatGroq initialized (model =", MODEL_NAME, ")")

ChatGroq initialized (model = deepseek-r1-distill-llama-70b )


In [ ]:
# HELPER: call Groq model
def run_llm(system_prompt: str, user_prompt: str) -> str:
    """Call the ChatGroq model and return the assistant's text content."""
    messages = [("system", system_prompt), ("human", user_prompt)]
    ai_msg = llm.invoke(messages)
    return ai_msg.content

In [ ]:
# STATE for LangGraph
class AgentState(TypedDict):
    text: str
    tools_to_run: List[str]       # optional extras, e.g. ["vocab"]
    tone: Optional[str]           # e.g. "formal"
    final_output: Optional[str]   # not strictly required, but kept for clarity

Setting up tools

The assistant uses four tools:

grammar_corrector – fixes grammar and punctuation.
Always runs first.

sentence_rewriter – improves clarity and flow.
Always runs after grammar correction.

vocab_enhancer – enriches word choice.
Runs only if the user requests vocabulary enhancement.

tone_adjuster – changes style (e.g., formal, casual).
Runs only if the user provides a tone.

Flow:
grammar → rewrite → (vocab if requested) → (tone if specified)

The LangGraph StateGraph enforces this order.
Grammar and rewriting are mandatory for every input; vocabulary and tone steps depend solely on the user’s optional choices.

In [ ]:
# NODES
def grammar_node(state: AgentState):
    system = "You are a meticulous grammar and punctuation editor. Return ONLY the corrected text."
    user   = f"Correct grammar, punctuation, and minor phrasing while preserving meaning. Text:\n\n{state['text']}"
    return {"text": run_llm(system, user)}

def rewrite_node(state: AgentState):
    system = "You are a helpful rewriter. Return ONLY the rewritten text focused on clarity and flow."
    user   = f"Rewrite the following to be clearer and better structured without changing meaning:\n\n{state['text']}"
    return {"text": run_llm(system, user)}

def vocab_node(state: AgentState):
    system = ("You are an editor who improves vocabulary and clarity. "
              "Return ONLY the improved text with better word choice while preserving meaning.")
    user   = f"Improve vocabulary and clarity:\n\n{state['text']}"
    return {"text": run_llm(system, user)}

def tone_node(state: AgentState):
    tone = state.get("tone", "neutral")
    system = f"You are a tone adjuster. Convert the text to a {tone} tone while preserving meaning. Return ONLY the adjusted text."
    user   = f"Adjust this text to {tone} tone:\n\n{state['text']}"
    return {"text": run_llm(system, user)}

In [ ]:
# BUILDING THE GRAPH
graph = StateGraph(AgentState)

graph.add_node("grammar", grammar_node)
graph.add_node("rewrite", rewrite_node)
graph.add_node("vocab", vocab_node)
graph.add_node("tone", tone_node)

graph.set_entry_point("grammar")
graph.add_edge("grammar", "rewrite")

# After rewrite, decide if vocab or tone is requested
graph.add_conditional_edges(
    "rewrite",
    lambda state: "vocab" if "vocab" in state.get("tools_to_run", []) else ("tone" if state.get("tone") else END),
    {"vocab": "vocab", "tone": "tone", END: END}
)

# After vocab, maybe tone
graph.add_conditional_edges(
    "vocab",
    lambda state: "tone" if state.get("tone") else END,
    {"tone": "tone", END: END}
)

graph.add_edge("tone", END)

app = graph.compile()


In [ ]:
# ===== UTIL =====
def strip_think_blocks(s: str) -> str:
    """Remove any <think>...</think> blocks that some models add."""
    return re.sub(r"<think>.*?</think>\s*", "", s, flags=re.DOTALL).strip()

# ===== CLI INTERACTION =====
user_text = input("Enter a sentence or paragraph: ").strip()
want_vocab = input("Add vocabulary enhancement? (y/n): ").strip().lower().startswith("y")
tone_req = input("Tone (leave blank for none, e.g. formal/casual): ").strip() or None

initial_state: AgentState = {
    "text": user_text,
    "tools_to_run": ["vocab"] if want_vocab else [],
    "tone": tone_req,
    "final_output": None
}

result = app.invoke(initial_state)
final_text = strip_think_blocks(result["text"])

print("\nImproved sentence:\n", final_text)

print("\nSteps executed:")
print(" - grammar_corrector")
print(" - sentence_rewriter")
if want_vocab:
    print(" - vocab_enhancer")
if tone_req:
    print(f" - tone_adjuster ({tone_req})")

Enter a sentence or paragraph: I am learnign ML it is fun.
Add vocabulary enhancement? (y/n): n
Tone (leave blank for none, e.g. formal/casual): casual

Improved sentence:
 I'm learning ML. It's fun!

Steps executed:
 - grammar_corrector
 - sentence_rewriter
 - tone_adjuster (casual)
